In [2]:
from pathlib import Path
import xarray as xr
import numpy as np

# in and out file paths 
original_data_dir = Path('data/original_data')

in_file = original_data_dir / 'CTD_all_1876-2019.nc'

transformed_data_dir = Path('data/transformed_data')

out_file = transformed_data_dir / 'CTD_all_1876-2019.nc'


#load in dataset
xrds = xr.open_dataset(in_file)

print(xrds)

<xarray.Dataset> Size: 13GB
Dimensions:      (PRES: 7225, TIME: 76708, CALIBRATION: 2)
Coordinates:
  * PRES         (PRES) float64 58kB 1.0 2.0 3.0 ... 7.224e+03 7.225e+03
  * TIME         (TIME) datetime64[ns] 614kB 1876-07-10T00:00:01.000004608 .....
    CALIBRATION  (CALIBRATION, TIME) float64 1MB ...
Data variables:
    STATION      (TIME) |S25 2MB ...
    LATITUDE     (TIME) float64 614kB ...
    LONGITUDE    (TIME) float64 614kB ...
    TEMP         (TIME, PRES) float64 4GB ...
    PSAL         (TIME, PRES) float64 4GB ...
    CNDC         (TIME, PRES) float64 4GB ...
    FDEP         (TIME) float64 614kB ...
    SHIP         (TIME) |S52 4MB ...
    SHIPID       (TIME) float32 307kB ...
    OWNER        (TIME) |S31 2MB ...
    CRUISE       (TIME) |S47 4MB ...
    TYPE         (TIME) |S47 4MB ...
Attributes: (12/38)
    cruise_ID:                     
    instrument:                    Shipborne observation
    institution:                   The University Centre in Svalbard, Nor

In [3]:


# Generate a unique integer CAST value for each time point
cast_values = np.arange(1, len(xrds['TIME']) + 1)

# Create a new CAST dimension with integer values
xrds = xrds.rename({'TIME': 'CAST'})

# Move the original TIME to a data variable that is CAST dependent
xrds['TIME'] = (('CAST',), xrds['CAST'].values)

# Assign integer CAST values
xrds['CAST'] = (('CAST',), cast_values)

# Modify the dimensions of data variables to be dependent on 'CAST' instead of 'TIME'
data_vars_to_reshape = [var for var in xrds.data_vars if 'CAST' in xrds[var].dims]

for var in data_vars_to_reshape:
    xrds[var] = xrds[var].rename({'CAST': 'CAST'})

# Save the modified dataset to a new NetCDF file
# xrds.to_netcdf('output.nc')
print(xrds['CAST'])
print(cast_values[:10])
print("The NetCDF file has been successfully modified and saved as 'output.nc'.")

<xarray.DataArray 'CAST' (CAST: 76708)> Size: 614kB
array([    1,     2,     3, ..., 76706, 76707, 76708])
Coordinates:
  * CAST     (CAST) int64 614kB 1 2 3 4 5 6 ... 76704 76705 76706 76707 76708
[ 1  2  3  4  5  6  7  8  9 10]
The NetCDF file has been successfully modified and saved as 'output.nc'.


In [4]:
for cast in range(1,11):
    time_value = xrds['TIME'].sel(CAST=cast).values
    print(time_value)

1876-07-10T00:00:01.000004608
1880-07-08T00:00:01.000004608
1890-07-02T00:00:01.000004608
1890-07-05T00:00:01.000004608
1890-07-08T00:00:01.000004608
1890-07-08T00:00:01.000004608
1890-07-08T00:00:01.000004608
1890-07-10T00:00:01.000004608
1890-07-10T00:00:01.000004608
1890-07-21T00:00:01.000004608


In [5]:
from check_for_met_ACDD import missing_MET_ACDD_attributes

transformed_data_dir = Path('data/transformed_data')

out_file = transformed_data_dir / 'CTD_all_1876-2019.nc'

missing_MET_ACDD_attributes(out_file)

data/transformed_data/CTD_all_1876-2019.nc is missing or has empty the following required attributes:
  - publisher_name: Required if not hosted by MET (MISSING)
  - publisher_email: Required if not hosted by MET (MISSING)
  - publisher_url: Required if not hosted by MET (MISSING)


In [6]:


missing_MET_ACDD_attributes('/home/alessioc/UNIS_hydrographic_data_to_cfnetcdf/data/transformed_data/CTD_1980-1989/UNIS_HD_19800106T034000_79_2.nc')

/home/alessioc/UNIS_hydrographic_data_to_cfnetcdf/data/transformed_data/CTD_1980-1989/UNIS_HD_19800106T034000_79_2.nc is missing or has empty the following required attributes:
  - publisher_name: Required if not hosted by MET (MISSING)
  - publisher_email: Required if not hosted by MET (MISSING)
  - publisher_url: Required if not hosted by MET (MISSING)


## Checking transformed_data agains CF-1.7 convetions

In [7]:
import os
import subprocess

subdirs = [d for d in transformed_data_dir.iterdir() if d.is_dir()]


unexpected_error_files = []

# Loop through each subdirectory
for subdir in subdirs:
    print(f"Subdirectory: {subdir.name}")
    
    # List .nc files in the current subdirectory
    nc_files = list(subdir.glob('*.nc'))
    
    tests_passed = 'All tests passed!'


    
    # Loop through each .nc file
    for nc_file in nc_files:

        command = f"compliance-checker --test=cf:1.7 {nc_file}"
        result = subprocess.run(command, shell=True, capture_output=True, text=True)

        # Check if all tests passed
        if tests_passed != result.stdout.strip().split('\n')[-1]:
            print(f'ERRORS FOUND in file {nc_file}!')
            unexpected_error_files.append(nc_file.name)
        

Subdirectory: CTD_2000-2009


KeyboardInterrupt: 

# Looking into removing CALIBRATION if empty

In [3]:
file = '/home/alessioc/UNIS_hydrographic_data_to_cfnetcdf/data/transformed_data/CTD_all_1876-2019.nc'

xrds = xr.open_dataset(file)

xrds

<xarray.Dataset> Size: 13GB
Dimensions:      (PRES: 7225, CAST: 76708, CALIBRATION_COEFFICIENT: 2)
Coordinates:
  * PRES         (PRES) float64 58kB 1.0 2.0 3.0 ... 7.224e+03 7.225e+03
  * CAST         (CAST) int32 307kB 1 2 3 4 5 ... 76704 76705 76706 76707 76708
    TIME         (CAST) datetime64[ns] 614kB ...
Dimensions without coordinates: CALIBRATION_COEFFICIENT
Data variables: (12/13)
    CALIBRATION  (CALIBRATION_COEFFICIENT, CAST) float64 1MB ...
    STATION      (CAST) |S25 2MB ...
    LATITUDE     (CAST) float64 614kB ...
    LONGITUDE    (CAST) float64 614kB ...
    TEMP         (CAST, PRES) float64 4GB ...
    PSAL         (CAST, PRES) float64 4GB ...
    ...           ...
    FDEP         (CAST) float64 614kB ...
    SHIP         (CAST) |S52 4MB ...
    SHIPID       (CAST) float32 307kB ...
    OWNER        (CAST) |S31 2MB ...
    CRUISE       (CAST) |S47 4MB ...
    TYPE         (CAST) |S47 4MB ...
Attributes: (12/43)
    cruise_ID:                     
    instrument:                    Shipborne observation
    institution:                   The University Centre in Svalbard, Norway
    project:                       A collection of projects have contributed ...
    data_assembly_center:          GOVERNMENT AGENCIES-NON-US>NORWAY>>>NO/UNIS
    naming_authority:              The University Centre in Svalbard, Norway
    ...                            ...
    date_created:                  2019-08-16T00:00:00Z
    creator_type:                  person
    creator_institution:           University Centre in Svalbard; Norwegian P...
    creator_name:                  Ragnheid Skogseth; Mathias Bockwoldt
    creator_email:                 Ragnheid.Skogseth@unis.no; mathias.bockwol...
    creator_url:                   https://orcid.org/0000-0003-0210-4981; htt...

In [46]:
# # Flatten the array for easy analysis
# flattened_arr = arr.flatten()

# # Check unique values in the array
# unique_values = np.unique(flattened_arr)

# # Check if all values are 0 or NaN
# all_zero_or_nan = np.all((flattened_arr == 0) | np.isnan(flattened_arr))

# print("Unique values in the array:", unique_values)
# print("All values are 0 or NaN:", all_zero_or_nan)

# # Identify non-zero and non-NaN values
# non_zero_and_non_nan_values = flattened_arr[(flattened_arr != 0) & ~np.isnan(flattened_arr)]

# # Count the number of such values
# count_non_zero_and_non_nan = len(non_zero_and_non_nan_values)

# print("Number of non-zero and non-NaN values:", count_non_zero_and_non_nan)


# # Identify NaN values
# nan_values = np.isnan(flattened_arr)

# # Identify zero values
# zero_values = flattened_arr == 0

# # Combine NaN and zero values
# nan_or_zero_values = nan_values | zero_values

# # Count the number of NaN or zero values
# count_nan_or_zero = np.sum(nan_or_zero_values)

# print("Number of NaN or 0 values:", count_nan_or_zero)

# print(np.argwhere(non_zero_and_non_nan_values))

Unique values in the array: [0.         0.9454     0.9691     0.9956     0.99974693 0.999769
 0.999783   0.999881   0.99998    1.         1.0000503  1.00011
 1.00013    1.00015    1.00024556 1.000267   1.00027854 1.0003183
 1.0006            nan]
All values are 0 or NaN: False
Number of non-zero and non-NaN values: 1890
Number of NaN or 0 values: 151526
[[   0]
 [   1]
 [   2]
 ...
 [1887]
 [1888]
 [1889]]


In [47]:
# # Create a boolean mask for non-NaN and non-zero values
# non_nan_non_zero_mask = (~np.isnan(arr)) & (arr != 0)

# # Find indices where mask is True
# indices = np.argwhere(non_nan_non_zero_mask)

# # Print the indices of non-NaN and non-zero values
# print("Indices of non-NaN and non-zero values:", indices)

Indices of non-NaN and non-zero values: [[    0 52838]
 [    0 52840]
 [    0 52841]
 ...
 [    0 67332]
 [    0 67333]
 [    0 67334]]


In [14]:
file = '/home/alessioc/UNIS_hydrographic_data_to_cfnetcdf/data/transformed_data/CTD_1876-1969/UNIS_HD_18900710T000000_78.5_16.2.nc'

xrds = xr.open_dataset(file)

xrds

<xarray.Dataset> Size: 840B
Dimensions:      (PRES: 50, TIME: 1, CALIBRATION_COEFFICIENT: 2)
Coordinates:
  * PRES         (PRES) float64 400B 1.0 2.0 3.0 4.0 5.0 ... 47.0 48.0 49.0 50.0
  * TIME         (TIME) datetime64[ns] 8B 1890-07-10T00:00:01.000004608
Dimensions without coordinates: CALIBRATION_COEFFICIENT
Data variables:
    CALIBRATION  (CALIBRATION_COEFFICIENT, TIME) float64 16B ...
    LATITUDE     (TIME) float64 8B ...
    LONGITUDE    (TIME) float64 8B ...
    TEMP         (TIME, PRES) float64 400B ...
Attributes: (12/45)
    cruise_ID:                     Gronfjord_1890_7
    instrument_type:               C
    instrument:                    Shipborne observation
    institution:                   The University Centre in Svalbard, Norway
    project:                       A collection of projects have contributed ...
    data_assembly_center:          GOVERNMENT AGENCIES-NON-US>NORWAY>>>NO/UNIS
    ...                            ...
    creator_type:                  person
    creator_institution:           University Centre in Svalbard; Norwegian P...
    creator_name:                  Ragnheid Skogseth; Mathias Bockwoldt
    creator_email:                 Ragnheid.Skogseth@unis.no; mathias.bockwol...
    creator_url:                   https://orcid.org/0000-0003-0210-4981; htt...
    featureType:                   profile